<a href="https://colab.research.google.com/github/sanaipei001/Data-Science-Excellerate-Internship-/blob/main/Team_K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

STEP 1: Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np

STEP 2: Load Dataset

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving SLU Opportunity Wise Data-1710158595043 - SLU Opportunity Wise Data-1710158595043.csv to SLU Opportunity Wise Data-1710158595043 - SLU Opportunity Wise Data-1710158595043.csv


In [ ]:
import io
import pandas as pd # Ensure pandas is imported here
df = pd.read_csv(io.BytesIO(uploaded['SLU Opportunity Wise Data-1710158595043 - SLU Opportunity Wise Data-1710158595043.csv']))

STEP 3: Handle Missing Values

In [ ]:
# View missing summary
print(" Initial missing values:\n", df.isnull().sum())

 Initial missing values:
 Learner SignUp DateTime       0
Opportunity Id                0
Opportunity Name              0
Opportunity Category          0
Opportunity End Date          0
First Name                    0
Date of Birth                 0
Gender                        0
Country                       0
Institution Name              5
Current/Intended Major        5
Entry created at              0
Status Description            0
Status Code                   0
Apply Date                    0
Opportunity Start Date     3794
dtype: int64


STEP 4: Expected Columns

In [ ]:
# Ensure all expected columns are present
expected_columns = [
    'Learner SignUp DateTime', 'Opportunity Id', 'Opportunity Name', 'Opportunity Category',
    'Date of Birth', 'Gender', 'Country', 'Institution Name', 'Major',
    'Apply Date', 'Opportunity Start Date', 'Opportunity End Date',
    'Status Description', 'Status Code', 'Entry created at'
]

In [ ]:
# Check and fill any missing columns
import numpy as np # Add import here for robustness
for col in expected_columns:
    if col not in df.columns:
        df[col] = np.nan  # Add missing column with NaN

In [ ]:
# Fill missing 'Institution Name' with 'Unknown'
df['Institution Name'] = df['Institution Name'].fillna('Unknown')

STEP 5: Convert Date Column to DateTime Format

In [ ]:
# Fix dates and convert
date_columns = [
    'Learner SignUp DateTime', 'Apply Date', 'Opportunity Start Date',
    'Opportunity End Date', 'Date of Birth', 'Entry created at'
]

for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

In [ ]:
# Fill Opportunity Start/End Date with placeholder if needed
df['Opportunity Start Date'] = df['Opportunity Start Date'].fillna(pd.to_datetime('1900-01-01'))
df['Opportunity End Date'] = df['Opportunity End Date'].fillna(pd.to_datetime('1900-01-01'))

# Fill Date of Birth with median date if missing
median_dob = df['Date of Birth'].median()
df['Date of Birth'] = df['Date of Birth'].fillna(median_dob)

# Fill any other missing categorical values with 'Unknown'
df = df.fillna('Unknown')

In [ ]:
# Final check: no missing values
print("\n Final missing values:\n", df.isnull().sum())


 Final missing values:
 Learner SignUp DateTime    0
Opportunity Id             0
Opportunity Name           0
Opportunity Category       0
Opportunity End Date       0
First Name                 0
Date of Birth              0
Gender                     0
Country                    0
Institution Name           0
Current/Intended Major     0
Entry created at           0
Status Description         0
Status Code                0
Apply Date                 0
Opportunity Start Date     0
Major                      0
dtype: int64


STEP 6: Featured Engineering

In [ ]:
# 1. Age
# Ensure Date of Birth is datetime before calculating age
df['Date of Birth'] = pd.to_datetime(df['Date of Birth'], errors='coerce')
df['Age'] = (pd.to_datetime('today') - df['Date of Birth']).dt.days // 365


# 2. Signup Month
# Ensure Learner SignUp DateTime is datetime before extracting month
df['Learner SignUp DateTime'] = pd.to_datetime(df['Learner SignUp DateTime'], errors='coerce')
# Extract month. .dt accessor handles NaT correctly, resulting in NaN
df['Signup Month'] = df['Learner SignUp DateTime'].dt.month
# Fill NaNs in 'Signup Month' if needed, for example, with 0 or a more appropriate value
df['Signup Month'] = df['Signup Month'].fillna(0) # Filling NaNs in month with 0

# 3. Application Lag
# Ensure Apply Date and Learner SignUp DateTime are datetime
df['Apply Date'] = pd.to_datetime(df['Apply Date'], errors='coerce')
df['Learner SignUp DateTime'] = pd.to_datetime(df['Learner SignUp DateTime'], errors='coerce')
df['Application Lag'] = (df['Apply Date'] - df['Learner SignUp DateTime']).dt.days
# Fill NaNs in 'Application Lag' if needed
df['Application Lag'] = df['Application Lag'].fillna(-1) # Filling NaNs in lag with -1

# 4. Opportunity Duration
# Ensure Opportunity End Date and Opportunity Start Date are datetime
df['Opportunity End Date'] = pd.to_datetime(df['Opportunity End Date'], errors='coerce')
df['Opportunity Start Date'] = pd.to_datetime(df['Opportunity Start Date'], errors='coerce')
df['Opportunity Duration (days)'] = (df['Opportunity End Date'] - df['Opportunity Start Date']).dt.days
# Fill NaNs in 'Opportunity Duration (days)' if needed
df['Opportunity Duration (days)'] = df['Opportunity Duration (days)'].fillna(-1) # Filling NaNs in duration with -1


# 5. Is International
df['Is International'] = df['Country'].apply(lambda x: False if str(x).strip().lower() == 'united states' else True)

# 6. Status Category
def categorize_status(code):
    try:
        # Convert code to string first to handle potential non-string types from previous fillna('Unknown')
        code_str = str(code)
        code_int = int(code_str)
        if code_int == 1080:
            return 'Started'
        elif code_int == 1070:
            return 'Team Allocated'
        else:
            return 'Other'
    except ValueError: # Catch errors during int conversion
        return 'Other'
    except TypeError: # Catch other potential type errors
         return 'Other'

df['Status Category'] = df['Status Code'].apply(categorize_status)

# Re-apply the fillna('Unknown') only for columns where 'Unknown' is appropriate
# Identify columns that should NOT be filled with 'Unknown' (e.g., numeric or datetime)
cols_to_exclude_from_unknown_fill = ['Age', 'Signup Month', 'Application Lag', 'Opportunity Duration (days)'] + date_columns

# Fill remaining NaNs with 'Unknown' only in columns where it makes sense
for col in df.columns:
    if col not in cols_to_exclude_from_unknown_fill:
        df[col] = df[col].fillna('Unknown')

# Handle potential NaNs in date columns if they weren't filled with a specific date placeholder before
# This step might be redundant if previous fillna for dates were sufficient, but included for robustness
for col in date_columns:
     if df[col].isnull().any():
         # Choose an appropriate fill value for date columns if needed, e.g., a minimum date
         df[col] = df[col].fillna(pd.to_datetime('1900-01-01'))

STEP 7: Verify Missing Values after Cleaning

In [ ]:
print("\n Missing values after cleaning:")
print(df.isnull().sum())


 Missing values after cleaning:
Learner SignUp DateTime        0
Opportunity Id                 0
Opportunity Name               0
Opportunity Category           0
Opportunity End Date           0
First Name                     0
Date of Birth                  0
Gender                         0
Country                        0
Institution Name               0
Current/Intended Major         0
Entry created at               0
Status Description             0
Status Code                    0
Apply Date                     0
Opportunity Start Date         0
Major                          0
Age                            0
Signup Month                   0
Application Lag                0
Opportunity Duration (days)    0
Is International               0
Status Category                0
dtype: int64


STEP 8: Export Cleaned Dataset

In [ ]:
# Final Cleaned Dataset
df.to_csv('/content/SLU_Cleaned_Final.csv', index=False)
print("\n Cleaned dataset exported as 'SLU_Cleaned_Final.csv'")


 Cleaned dataset exported as 'SLU_Cleaned_Final.csv'


STEP 9: Download CSV Cleaned Dataset File

In [ ]:
#  Download
files.download('/content/SLU_Cleaned_Final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>